# Resnet

## Please watch Ng C4W2L01-C4W2L04, the first of which is found [here](https://www.youtube.com/watch?v=-bvTzZCEOdM&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&index=12).

The convolutional neural network that we developed and ran was adequate for use on a small problem with a few classes, but it lacks the explanatory power to produce highly accurate results for more difficult datasets.  Instead, more interesting neural networks have been developed which have greater explanatory power.  One of the most powerful architectures today is called ResNet, which is short for residual network.  

In principle, you could take the network that you've been working on and make it more flexible by adding more convolutional layers, which is to say that we could add more sequences of feature map generation.  This is what is meant when people use the term "deep" learning.  However, if you did this, you would quickly run into the problem that your network would struggle to learn weights in the lower (closer to the inputs) layers of the network.  This is a result of the way that neural networks are trained.  In particular they rely on the ability to take the derivative of a misfit function (e.g. least squares) with respect to a parameter, and to adjust the weight based on that derivative.  However in (naive) deep networks, this gradient has the tendency to become negligibly small as the impact of that weight gets lost in the myriad layers of convolutions and activations closer to the output.  

ResNet solves this problem by ensuring that the information in each weight gets propagated to the output.  It does this by simply adding the layer's input to each layer's output, so instead of 
$$
\mathbf{x}_{l+1} = \mathcal{F}_{l}(\mathbf{x}_l),
$$
at each layer, the neural network performs the operation
$$
\mathbf{x}_{l+1} = \mathcal{F}_{l}(\mathbf{x}_l) + \mathbf{x}_l.
$$
Rearranging this equation, we can see why this architecture is called a residual network:
$$
\mathbf{x}_{l+1} - \mathbf{x}_l = \mathcal{F}_{l}(\mathbf{x}_l).
$$
Each layer is modeling the residual between consecutive feature maps.  The pedantic amongst us will note that this only works when the output of $\mathcal{F}_{l}(\mathbf{x}_l)$ is the same size as the input.  This is dealt with by performing a suitable linear transformation on $\mathbf{x}_l$, making the equation
$$
\mathbf{x}_{l+1} = \mathcal{F}_{l}(\mathbf{x}_l) + W \mathbf{x}_l,
$$
where $W$ is a matrix that has learnable weights.  The matrix $W$ is most often formulated as a convolution with a 1x1 kernel size.   

The addition of the input is known as a *skip connection* because it looks like this:
<img src=res_net.svg width=600/>
The input is run through a normal conv layer (perhaps several) and then added to the output, where it can then be maxpooled or run through an activation or whatever.  

Keras makes these sorts of networks pretty easy to program.  To start with, let's apply this network to the CIFAR-10 classification problem, but we'll do it for all 10 classes.  All the non-model definition code should look the same as our previous example.   

In [3]:
import keras
import keras.datasets as kd

(x_train, y_train), (x_test, y_test) = kd.cifar10.load_data()
labels = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

x_train = x_train/255.
x_test = x_test/255.

# Convert class vectors to binary class matrices.
N = len(labels)

y_train = keras.utils.to_categorical(y_train, N)
y_test = keras.utils.to_categorical(y_test, N)

Using TensorFlow backend.


Now things get more interesting.  Obviously, ResNet as described above is more of a concept than a specific architecture: we'll need to make some more specific design choices.  One good way of doing this is to look at the literature and copy what others have done.  In particular, the [original ResNet Paper](https://arxiv.org/abs/1512.03385) provides an example of ResNet being applied to CIFAR-10 that yielded excellent accuracy (state of the art c. 2015).  Here, we'll emulate their network architecture, which looks like this:
<img src=cifar_10_res_net.svg width=900/>
More concretely, the layers of this network up to (and including) the location of the star in the figure above, look like this.

In [4]:
import keras.layers as kl
import keras.regularizers as kr

# Note the alternative method for model specification: no model.add(.), instead we 
# perform sequential operations on layers, then we will make the resulting model later.

# Specify the shape of the input image
input_shape = x_train.shape[1:]
inputs = kl.Input(shape=input_shape)

# First convolution + BN + act
conv = kl.Conv2D(16,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(inputs)
bn = kl.BatchNormalization()(conv)
act1 = kl.Activation('relu')(bn)

# Perform 3 convolution blocks
for i in range(3):
    conv = kl.Conv2D(16,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act1)
    bn = kl.BatchNormalization()(conv)
    act = kl.Activation('relu')(bn)
    
    conv = kl.Conv2D(16,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act)
    bn = kl.BatchNormalization()(conv)

    # Skip layer addition
    skip = kl.add([act1,bn])
    act1 = kl.Activation('relu')(skip)  

# Downsampling with strided convolution
conv = kl.Conv2D(32,(3,3),padding='same',strides=2,kernel_regularizer=kr.l2(1e-4))(act1)
bn = kl.BatchNormalization()(conv)
act = kl.Activation('relu')(bn)

conv = kl.Conv2D(32,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act)
bn = kl.BatchNormalization()(conv)

# Downsampling with strided 1x1 convolution
act1_downsampled = kl.Conv2D(32,(1,1),padding='same',strides=2,kernel_regularizer=kr.l2(1e-4))(act1)

# Downsampling skip layer
skip_downsampled = kl.add([act1_downsampled,bn])
act1 = kl.Activation('relu')(skip_downsampled)

#2x identity 
for i in range(2):
    conv = kl.Conv2D(32,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act1)
    bn = kl.BatchNormalization()(conv)
    act = kl.Activation('relu')(bn)
    
    conv = kl.Conv2D(32,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act)
    bn = kl.BatchNormalization()(conv)

    # Skip layer addition
    skip = kl.add([act1,bn])
    act1 = kl.Activation('relu')(skip)
    
conv = kl.Conv2D(64,(3,3),padding='same',strides=2,kernel_regularizer=kr.l2(1e-4))(act1)
bn = kl.BatchNormalization()(conv)
act = kl.Activation('relu')(bn)

conv = kl.Conv2D(64,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act)
bn = kl.BatchNormalization()(conv)

# Downsampling with strided 1x1 convolution
act1_downsampled = kl.Conv2D(64,(1,1),padding='same',strides=2,kernel_regularizer=kr.l2(1e-4))(act1)

# Downsampling skip layer
skip_downsampled = kl.add([act1_downsampled,bn])
act1 = kl.Activation('relu')(skip_downsampled)

#2x identity 
for i in range(2):
    conv = kl.Conv2D(64,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act1)
    bn = kl.BatchNormalization()(conv)
    act = kl.Activation('relu')(bn)
    
    conv = kl.Conv2D(64,(3,3),padding='same',kernel_regularizer=kr.l2(1e-4))(act)
    bn = kl.BatchNormalization()(conv)

    # Skip layer addition
    skip = kl.add([act1,bn])
    act1 = kl.Activation('relu')(skip)

# This final layer is denoted by a star in the above figure

Instructions for updating:
Colocations handled automatically by placer.


To ensure that we have the output shape that we expect at this stage, we can look at the shape of act1  

In [3]:
act1

<tf.Tensor 'activation_19/Relu:0' shape=(?, 8, 8, 64) dtype=float32>

Which is an object of size 16x16x32, the correct size based on our chosen architecture (note the first question mark indicates an unknown number of input images: thus if we ran the model on a single photo, this would be a 1, if we ran it on the entire CIFAR training set at once it would be 50000).  As before, we can use this model for classification by doing global average pooling, then the softmax function.

In [4]:
gap = kl.GlobalAveragePooling2D()(act1)
bn = kl.BatchNormalization()(gap)
final_dense = kl.Dense(N)(bn)
softmax = kl.Activation('softmax')(final_dense)

In [5]:
import keras.models as km
model = km.Model(inputs=inputs,outputs=softmax)

print(model.summary())

# initiate adam optimizer
opt = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=25,
          validation_data=(x_test, y_test),
          shuffle=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 305s 6ms/step - loss: 1.4780 - acc: 0.4899 - val_loss: 1.8339 - val_acc: 0.4278
Epoch 2/25
50000/50000 [==============================] - 307s 6ms/step - loss: 1.0362 - acc: 0.6603 - val_loss: 1.2745 - val_acc: 0.6029
Epoch 3/25
50000/50000 [==============================] - 308s 6ms/step - loss: 0.8502 - acc: 0.7314 - val_loss: 1.2445 - val_acc: 0.5843
Epoch 4/25
50000/50000 [==============================] - 304s 6ms/step - loss: 0.7404 - acc: 0.7712 - val_loss: 1.0205 - val_acc: 0.6948
Epoch 5/25
50000/50000 [==============================] - 301s 6ms/step - loss: 0.6575 - acc: 0.8002 - val_loss: 1.0101 - val_acc: 0.7025
Epoch 6/25
50000/50000 [==============================] - 300s 6ms/step - loss: 0.5919 - acc: 0.8255 - val_loss: 1.0084 - val_acc: 0.7044
Epoch 7/25
50000/50000 [==============================] - 300s 6ms/step - 

While the code as is works, it is *not* the complete architecture given in the figure above.  **Implement the remainder of the network, and train the model for 100 epochs.** The complete architecture has quite a few parameters, so you'll definitely want to use a GPU, i.e. run it on the cluster (reference the job script included in this repo).

There are also a few extra tidbits to make this work better.  First, we'll want to checkpoint the model, which is to say that we'll want to save the weights anytime the model improves during the training process.  We can do this easily in Keras with a checkpoint function:

In [6]:
import keras.callbacks as kc
filepath = './checkpoints'

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = kc.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

Note that these weights can then be loaded into a model on your local machine for more convenient post-processing and visualization of results.  We'll also want to reduce the learning rate as the model reaches an optimal solution.  We can do this with a *learning rate schedule*.


In [22]:
def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 10:
        lr = 1e-5
    if epoch > 50:
        lr = 1e-6
    if epoch > 70:
        lr = 1e-7
    print('Learning rate: ', lr)
    return lr
lr_scheduler = kc.LearningRateScheduler(lr_schedule)

We can include these two functions as *callbacks* to the optimizer:

In [24]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=100,
          validation_data=(x_test, y_test),
          shuffle=True,
          callbacks=[checkpoint,lr_scheduler])

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
Learning rate:  0.0001
50000/50000 [==============================] - 388s 8ms/step - loss: 0.0506 - acc: 0.9948 - val_loss: 1.4699 - val_acc: 0.8026

Epoch 00001: val_acc did not improve from 0.81960
Epoch 2/100
Learning rate:  0.0001
50000/50000 [==============================] - 386s 8ms/step - loss: 0.0543 - acc: 0.9933 - val_loss: 1.4066 - val_acc: 0.8005

Epoch 00002: val_acc did not improve from 0.81960
Epoch 3/100
Learning rate:  0.0001
50000/50000 [==============================] - 419s 8ms/step - loss: 0.0555 - acc: 0.9926 - val_loss: 1.4277 - val_acc: 0.8049

Epoch 00003: val_acc did not improve from 0.81960
Epoch 4/100
Learning rate:  0.0001
50000/50000 [==============================] - 418s 8ms/step - loss: 0.0538 - acc: 0.9938 - val_loss: 1.2930 - val_acc: 0.8080

Epoch 00004: val_acc did not improve from 0.81960
Epoch 5/100
Learning rate:  0.0001
50000/50000 [==============================] - 343s 7ms/step - 

50000/50000 [==============================] - 342s 7ms/step - loss: 0.0288 - acc: 0.9999 - val_loss: 1.3131 - val_acc: 0.8170

Epoch 00076: val_acc did not improve from 0.82160
Epoch 77/100
Learning rate:  1e-07
50000/50000 [==============================] - 341s 7ms/step - loss: 0.0287 - acc: 0.9999 - val_loss: 1.3081 - val_acc: 0.8173

Epoch 00077: val_acc did not improve from 0.82160
Epoch 78/100
Learning rate:  1e-07
50000/50000 [==============================] - 341s 7ms/step - loss: 0.0288 - acc: 0.9998 - val_loss: 1.3108 - val_acc: 0.8174

Epoch 00078: val_acc did not improve from 0.82160
Epoch 79/100
Learning rate:  1e-07
50000/50000 [==============================] - 341s 7ms/step - loss: 0.0286 - acc: 1.0000 - val_loss: 1.3102 - val_acc: 0.8182

Epoch 00079: val_acc did not improve from 0.82160
Epoch 80/100
Learning rate:  1e-07
50000/50000 [==============================] - 341s 7ms/step - loss: 0.0289 - acc: 0.9999 - val_loss: 1.3100 - val_acc: 0.8171

Epoch 00080: val_acc

Once your model is fitted, **adapt your class activation mapping routine to run on this more advanced architecture, and compute a few examples?  How do these activation maps differ from those computed for the smaller network?**

In [25]:
model.evaluate(x=x_test, y=y_test)

10000/10000 [==============================] - 15s 2ms/step


[1.3103056852817536, 0.8179]

In [6]:
from keras.models import load_model

model = load_model('checkpoints')

Instructions for updating:
Use tf.cast instead.


In [7]:

import keras.callbacks as kc
filepath = './checkpoints'

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = kc.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 60:
        lr *= 1e-1
    elif epoch > 120:
        lr *= 1e-2
    print('Learning rate: ', lr)
    return lr
lr_scheduler = kc.LearningRateScheduler(lr_schedule)

from keras.preprocessing.image import ImageDataGenerator
    
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.01,  # set range for random shear
    zoom_range=0.2,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

batch_size = 64
epochs = 180

# Fit the model on the batches generated bydatagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                batch_size=batch_size),
                                steps_per_epoch=len(x_train)/batch_size,
                                epochs=epochs,
                                validation_data=(x_test, y_test),verbose=1, workers=4,
                                callbacks=[checkpoint,lr_scheduler])

Epoch 1/180
Learning rate:  0.001
782/781 [==============================] - 346s 443ms/step - loss: 0.9042 - acc: 0.7257 - val_loss: 1.9054 - val_acc: 0.5922

Epoch 00001: val_acc improved from -inf to 0.59220, saving model to ./checkpoints
Epoch 2/180
Learning rate:  0.001
782/781 [==============================] - 326s 417ms/step - loss: 0.7297 - acc: 0.7683 - val_loss: 1.4001 - val_acc: 0.6128

Epoch 00002: val_acc improved from 0.59220 to 0.61280, saving model to ./checkpoints
Epoch 3/180
Learning rate:  0.001
782/781 [==============================] - 326s 416ms/step - loss: 0.6784 - acc: 0.7883 - val_loss: 0.9798 - val_acc: 0.7143

Epoch 00003: val_acc improved from 0.61280 to 0.71430, saving model to ./checkpoints
Epoch 4/180
Learning rate:  0.001
782/781 [==============================] - 330s 422ms/step - loss: 0.6498 - acc: 0.8032 - val_loss: 1.0449 - val_acc: 0.7211

Epoch 00004: val_acc improved from 0.71430 to 0.72110, saving model to ./checkpoints
Epoch 5/180
Learning ra

782/781 [==============================] - 327s 419ms/step - loss: 0.4762 - acc: 0.8899 - val_loss: 0.8416 - val_acc: 0.8039

Epoch 00038: val_acc did not improve from 0.84390
Epoch 39/180
Learning rate:  0.001
782/781 [==============================] - 326s 416ms/step - loss: 0.4802 - acc: 0.8885 - val_loss: 0.7069 - val_acc: 0.8281

Epoch 00039: val_acc did not improve from 0.84390
Epoch 40/180
Learning rate:  0.001
782/781 [==============================] - 323s 413ms/step - loss: 0.4744 - acc: 0.8902 - val_loss: 1.1762 - val_acc: 0.7409

Epoch 00040: val_acc did not improve from 0.84390
Epoch 41/180
Learning rate:  0.001
782/781 [==============================] - 323s 412ms/step - loss: 0.4788 - acc: 0.8886 - val_loss: 0.6450 - val_acc: 0.8416

Epoch 00041: val_acc did not improve from 0.84390
Epoch 42/180
Learning rate:  0.001
782/781 [==============================] - 324s 414ms/step - loss: 0.4740 - acc: 0.8914 - val_loss: 0.8884 - val_acc: 0.7781

Epoch 00042: val_acc did not i

782/781 [==============================] - 328s 419ms/step - loss: 0.2931 - acc: 0.9476 - val_loss: 0.4645 - val_acc: 0.9024

Epoch 00075: val_acc did not improve from 0.90860
Epoch 76/180
Learning rate:  0.0001
782/781 [==============================] - 327s 418ms/step - loss: 0.2901 - acc: 0.9487 - val_loss: 0.4572 - val_acc: 0.9024

Epoch 00076: val_acc did not improve from 0.90860
Epoch 77/180
Learning rate:  0.0001
782/781 [==============================] - 327s 418ms/step - loss: 0.2893 - acc: 0.9488 - val_loss: 0.4719 - val_acc: 0.9001

Epoch 00077: val_acc did not improve from 0.90860
Epoch 78/180
Learning rate:  0.0001
782/781 [==============================] - 328s 419ms/step - loss: 0.2858 - acc: 0.9494 - val_loss: 0.4702 - val_acc: 0.9002

Epoch 00078: val_acc did not improve from 0.90860
Epoch 79/180
Learning rate:  0.0001
782/781 [==============================] - 329s 420ms/step - loss: 0.2808 - acc: 0.9508 - val_loss: 0.4825 - val_acc: 0.9000

Epoch 00079: val_acc did n


Epoch 00113: val_acc did not improve from 0.90860
Epoch 114/180
Learning rate:  0.0001
782/781 [==============================] - 325s 416ms/step - loss: 0.2249 - acc: 0.9618 - val_loss: 0.4485 - val_acc: 0.9059

Epoch 00114: val_acc did not improve from 0.90860
Epoch 115/180
Learning rate:  0.0001
782/781 [==============================] - 325s 416ms/step - loss: 0.2232 - acc: 0.9619 - val_loss: 0.4561 - val_acc: 0.9054

Epoch 00115: val_acc did not improve from 0.90860
Epoch 116/180
Learning rate:  0.0001
782/781 [==============================] - 325s 416ms/step - loss: 0.2202 - acc: 0.9619 - val_loss: 0.4604 - val_acc: 0.9049

Epoch 00116: val_acc did not improve from 0.90860
Epoch 117/180
Learning rate:  0.0001
782/781 [==============================] - 326s 416ms/step - loss: 0.2244 - acc: 0.9610 - val_loss: 0.4704 - val_acc: 0.9010

Epoch 00117: val_acc did not improve from 0.90860
Epoch 118/180
Learning rate:  0.0001
782/781 [==============================] - 325s 416ms/step -

782/781 [==============================] - 313s 400ms/step - loss: 0.1891 - acc: 0.9677 - val_loss: 0.4616 - val_acc: 0.9049

Epoch 00152: val_acc did not improve from 0.90860
Epoch 153/180
Learning rate:  0.0001
782/781 [==============================] - 313s 400ms/step - loss: 0.1883 - acc: 0.9688 - val_loss: 0.4501 - val_acc: 0.9037

Epoch 00153: val_acc did not improve from 0.90860
Epoch 154/180
Learning rate:  0.0001
782/781 [==============================] - 314s 402ms/step - loss: 0.1867 - acc: 0.9687 - val_loss: 0.4743 - val_acc: 0.9078

Epoch 00154: val_acc did not improve from 0.90860
Epoch 155/180
Learning rate:  0.0001
782/781 [==============================] - 313s 401ms/step - loss: 0.1860 - acc: 0.9685 - val_loss: 0.4673 - val_acc: 0.9058

Epoch 00155: val_acc did not improve from 0.90860
Epoch 156/180
Learning rate:  0.0001
782/781 [==============================] - 314s 401ms/step - loss: 0.1839 - acc: 0.9695 - val_loss: 0.4422 - val_acc: 0.9099

Epoch 00156: val_acc i